# 입력 특성 전처리하기
신경망을 위해 데이터를 준비하려면 일반적으로 모든 특성은 **수치 특성으로 변환하고 정규화해야 함**  
>특히 범주형 특성이나 텍스트 특성이 있다면 숫자로 바꿔야 함.

어떤 도구라도 이용해서(넘파이, 판다스, 사이킷런) 데이터 파일을 준비하기 전에 처리해야 함. 아니면 **데이터 API** 로 적재할 때 동적으로 전처리할 수도 있음.  
> 또는 **전처리 층** 을 따로 만들어서 모델에 직접 포함시킬 수도 있음.

In [1]:
import os
import tarfile
import urllib.request

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    os.makedirs(housing_path, exist_ok=True)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

In [2]:
fetch_housing_data()

In [3]:
import pandas as pd

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

In [4]:
housing = load_housing_data()
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [5]:
housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


---
## 1. 정규화
### 1.1 전역변수 사용
>각 특성의 평균, 표준편차를 미리 계산하여 전처리하는 방법

In [6]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)
train_num_set = train_set.drop("ocean_proximity", axis=1)
train_cat_set = train_set[["ocean_proximity"]]
train_num_set.shape, train_cat_set.shape

((16512, 9), (16512, 1))

In [7]:
import numpy as np

means = np.mean(train_num_set, axis=0)
stds = np.std(train_num_set, axis=0)
print(means.shape)
print("")
print(stds.shape)

(9,)

(9,)


In [8]:
scaled_train_num_set = (train_num_set - means) / stds
scaled_train_num_set.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
14196,1.272587,-1.372811,0.348490,0.222569,0.211228,0.768276,0.322906,-0.326196,-0.901189
8267,0.709162,-0.876696,1.618118,0.340293,0.593094,-0.098901,0.672027,-0.035843,1.512771
17445,-0.447603,-0.460146,-1.952710,-0.342597,-0.495226,-0.449818,-0.430461,0.144701,-0.299213
14265,1.232698,-1.382172,0.586545,-0.561490,-0.409306,-0.007434,-0.380587,-1.017864,-0.984220
2271,-0.108551,0.532084,1.142008,-0.119565,-0.256559,-0.485877,-0.314962,-0.171488,-0.957408


---
### 1.2 사용자 정의 스케일링 층
전역변수를 다루기보다 사이킷런의 StandardScaler 처럼 사용자 정의 층을 정의할 수도 있음.

In [9]:
from tensorflow import keras

class StandardizationLayer(keras.layers.Layer):
    def adapt(self, data_sample):
        self.means_ = np.mean(data_sample, axis=0)
        self.stds_ = np.std(data_sample, axis=0)
    
    def call(self, inputs):
        reutnr (inputs - self.means_) / (self.stds_ + keras.backend.epsilon())

In [10]:
std_layer = StandardizationLayer()
std_layer.adapt(train_set)

In [11]:
model = keras.Sequential()
model.add(std_layer)
model.compile(loss="mse", optimizer="sgd")

물론 위에 만든 것과 동일한 역할을 해주는 층이 이미 존재함.  
**keras.layers.Normalization** 층을 사용하면 됨.

---
## 2. 원-핫 인코딩
ocean_proximity 특성의 경우 "<1H OCEAN", "INLAND", "NEAR OCEAN", "NEAR BAY", "ISLAND" 다섯 개의 값이 가능한 **범주형 특성**임  
이 특성을 신경망에 주입하기 전에 인코딩해야 함.  
> 범주 개수가 작으므로 **원-핫 인코딩** 을 사용할 수 있음.

먼저 **룩업 테이블** 을 사용하여 각 범주를 인덱스(0 ~ 4)로 매핑해야 함

In [12]:
import tensorflow as tf

vocab = [key[0] for key in train_cat_set.value_counts().keys()]
indices = tf.range(len(vocab), dtype=tf.int64)
table_init = tf.lookup.KeyValueTensorInitializer(vocab, indices)
num_oov_buckets = 2
table = tf.lookup.StaticVocabularyTable(table_init, num_oov_buckets)

In [13]:
table

- **vocab** : 먼저 **어휘 사전** 을 정의함. 가능한 모든 범주의 리스트
- **indices** : 그 다음 범주에 해당하는 인덱스(0 ~ 4까지)의 텐서를 만듦.
- **table_init** : 그 다음 범주 리스트와 해당 인덱스를 전달하여 **룩업 테이블**의 초기화 객체를 만듦. 이 예에서는 이미 이 데이터를 갖고 있으므로 **KeyValueTensorInitializer** 를 사용함. (만약 범주가 텍스트 파일에 라인당 하나의 범주로 나열되어 있다면 **TextFileInitializer** 사용)
- **num_oob_buckets**, **table** : 초기화 객체와 **oov(out of vocabulary) 버킷** 을 지정하여 룩업 테이블을 만듦.  
    > 어휘 사전에 없는 범주를 찾으면 룩업 테이블이 계산한 이 범주와 해시값을 이용하여 oov버킷 중 하나에 할당함. 
    
    인덱스는 알려진 범주 다음부터 시작함. 따라서 이 예제에서 두 개의 oov버킷의 인덱스는 5와 6이 됨.

**oov 버킷** 은 왜 사용하는가?
범주 개수가 많고(우편번호, 도시, 단어, 상품, 사용자 등), 데이터셋이 크거나 범주가 자주 바뀐다면 전체 범주 리스트를 구하는 것이 어려울 수 있음.  
> 한 가지 해결책은 샘플 데이터를 기반으로 어휘 사전을 정의하고 샘플 데이터에 없는 다른 범주를 oov 버킷에 추가하는 것임.  

학습 도중에 발견되는 알려지지 않은 범주가 많을수록 더 많은 oov버킷을 사용해야 함.  
실제 oov버킷이 충분하지 않으면 충돌이 발생할 수도 있음. 즉 다른 범주가 동일한 버킷에 할당되는 것. 따라서 신경망은 두 범주를 구분할 수 없을 것.


이제 **룩업 테이블을 이용해서 몇 개의 범주 특성을 원-핫 인코딩** 해보겠음

In [14]:
sample_cat = tf.constant(["NEAR BAY", "DESERT", "INLAND", "INLAND"])
sample_cat_indices = table.lookup(sample_cat)
sample_cat_indices

<tf.Tensor: shape=(4,), dtype=int64, numpy=array([3, 5, 1, 1], dtype=int64)>

In [16]:
sample_cat_onehot = tf.one_hot(sample_cat_indices, depth=len(vocab) + num_oov_buckets)
sample_cat_onehot

<tf.Tensor: shape=(4, 7), dtype=float32, numpy=
array([[0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.]], dtype=float32)>

사전에 몰랐던 **DESERT** 의 경우 **oov_buckets** 로 설정한 인덱스인 (5, 6) 중 하나인 5로 매핑됨.  
그 다음 **tf.one_hot()** 함수를 사용하여 이 인덱스들을 원-핫 인코딩할 수 있음.  
이 함수는 어휘 사전 크기 + oov 버킷수를 더한 총 인덱스 개수를 지정해야 함.

> 물론 이것과 동일한 역할을 해주는 **keras.layers.TextVectorization** 층이 있음.  
**adapt()** 메서드가 샘플 데이터에서 어휘 사전을 추출하고  
**call()** 메서드가 각 범주를 어휘 사전에 있는 인덱스로 변환해줌.  
인덱스를 원-핫 벡터로 바꾸고 싶다면 이 층을 모델의 시작 부분에 추가하고 뒤이어 tf.one_hot() 함수가 적용된 lambda 층을 놓으면 됨

---
## 3. 임베딩을 통한 범주형 특성 인코딩
원-핫 벡터의 크기는 워휘 사전의 길이와 oov 버킷 개수를 더한 것임.  
- **가능한 범주가 10개 이하라면** : 원-핫 인코딩
- **가능한 범주가 50개 이상이라면** : **임베딩**
- 그 중간이면 실험을 통해 좋은거 쓰기

**임베딩**은 범주를 표현하는 학습 가능한 밀집 벡터를 뜻함.  
처음엔 임베딩이 랜덤하게 초기화됨. 비슷한 범주들은 경사 하강법이 더 가깝게 만들고 다른 범주들은 더 멀어지게 함.  
따라서 범주가 유용하게 표현되도록 임베딩이 학습하는 경향이 있음. 이를 **표현 학습**이라고 함.

임베딩을 직접 구현해 보겠음.  
먼저 각 범주의 임베딩을 담은 **임베딩 행렬** 을 만들어 랜덤하게 초기화해야 함.  
이 행렬은 범주와 oov 버킷마다 하나의 행이 있고 임베딩 차원마다 하나의 열을 가짐.

In [17]:
embedding_dim = 2
embed_init = tf.random.uniform([len(vocab) + num_oov_buckets, embedding_dim])
embedding_matrix = tf.Variable(embed_init)
embedding_matrix

<tf.Variable 'Variable:0' shape=(7, 2) dtype=float32, numpy=
array([[0.5881648 , 0.23527181],
       [0.25686812, 0.3070649 ],
       [0.93952024, 0.8801329 ],
       [0.38930273, 0.90594923],
       [0.09404457, 0.84525514],
       [0.7684406 , 0.8395574 ],
       [0.57339346, 0.62286174]], dtype=float32)>

이제 **이 임베딩을 사용해 앞에서와 동일한 범주 특성을 인코딩해보겠음**

In [18]:
sample_cat = tf.constant(["NEAR BAY", "DESERT", "INLAND", "INLAND"])
sample_cat_indices = table.lookup(sample_cat)
sample_cat_indices

<tf.Tensor: shape=(4,), dtype=int64, numpy=array([3, 5, 1, 1], dtype=int64)>

In [19]:
sample_cat_emb = tf.nn.embedding_lookup(embedding_matrix, sample_cat_indices)
sample_cat_emb

<tf.Tensor: shape=(4, 2), dtype=float32, numpy=
array([[0.38930273, 0.90594923],
       [0.7684406 , 0.8395574 ],
       [0.25686812, 0.3070649 ],
       [0.25686812, 0.3070649 ]], dtype=float32)>

> 역시 위와 같은 역할을 해주는 **keras.layers.Embedding** 층이 있음.  
이 층이 생성될 때 임베딩 행렬을 랜덤하게 초기화하고 어떤 범주 인덱스로 호출될 때 임베딩 핼령에 있는 그 인덱스의 행을 반환함.

In [20]:
embedding_layer = keras.layers.Embedding(input_dim=len(vocab)+num_oov_buckets,
                                        output_dim=embedding_dim)
embedding_layer(sample_cat_indices)

<tf.Tensor: shape=(4, 2), dtype=float32, numpy=
array([[-0.00108191, -0.00264404],
       [ 0.02433498,  0.0383966 ],
       [ 0.03332975,  0.0418602 ],
       [ 0.03332975,  0.0418602 ]], dtype=float32)>

>앞서 했던 모든 것을 연결하면  
범주형 특성을 처리하고 각 범주(그리고 각 oov 버킷)마다 임베딩을 학습하는 케라스 모델을 만들 수 있음.

In [24]:
regular_inputs = keras.layers.Input(shape=[8])

cat = keras.layers.Input(shape=[], dtype=tf.string)
cat_indices = keras.layers.Lambda(lambda cats: table.lookup(cats))(cat)
cat_embed = keras.layers.Embedding(input_dim=6, output_dim=2)(cat_indices)

encoded_inputs = keras.layers.concatenate([regular_inputs, cat_embed])

outputs = keras.layers.Dense(1)(encoded_inputs)

model = keras.models.Model(inputs=[regular_inputs, cat], outputs=[outputs])

cat_model = keras.models.Model(inputs=[cat], outputs=[cat_embed])

In [25]:
cat_model.predict(sample_cat)

array([[ 0.02444382,  0.02794207],
       [-0.00798589,  0.02676889],
       [-0.0221585 ,  0.03709747],
       [-0.0221585 ,  0.03709747]], dtype=float32)

**keras.layers.TextVectorization** 층을 사용하면 **adapt()** 메서드를 호출하여 샘플 데이터에서 어휘 사전을 추출하는 것으로 룩업 테이블을 만들 수 있음.  
그 다음 이 층을 모델에 추가하여 인덱스 룩업을 수행할 것(Lambda 층을 대신하게 됨)

In [51]:
tv_layer = keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=10,
    output_mode="int",
    output_sequence_length = 2,)
tv_layer.adapt(vocab)

sample_cat2 = tf.constant(["NEAR BAY","INLAND", "INLAND", "NEAR OCEAN"])

tv_layer.get_vocabulary()

['', '[UNK]', 'ocean', 'near', 'island', 'inland', 'bay', '1h']

In [52]:
sample_cat2

<tf.Tensor: shape=(4,), dtype=string, numpy=array([b'NEAR BAY', b'INLAND', b'INLAND', b'NEAR OCEAN'], dtype=object)>

In [53]:
vocab

['<1H OCEAN', 'INLAND', 'NEAR OCEAN', 'NEAR BAY', 'ISLAND']

In [54]:
tv_layer(sample_cat2)

<tf.Tensor: shape=(4, 2), dtype=int64, numpy=
array([[3, 6],
       [5, 0],
       [5, 0],
       [3, 2]], dtype=int64)>

---
## 케라스 전처리 층
- **keras.layers.Normalization** : 특성 표준화를 수행
- **TextVectorization** : 입력에 있는 각 단어를 어휘 사전에 있는 인덱서로 인코딩

두 경우 모두 층을 만들고 샘플 데이터로 **adapt()** 메서드를 호출한 다음 일반 층처럼 모델에 사용할 수 있음.  
다른 전처리 층들도 동일한 패턴을 따름

>**keras.layers.experimental.preprocessing.Discretization()** 층은 연속적인 데이터를 몇 개의 구간으로 나누고 각 구간을 원-핫 벡터로 인코딩함. 예를 들어 가격 데이터 (낮음, 중간, 높음)은 ([1,0,0], [0,1,0], [0,0,1])로 인코딩됨.  
이렇게 하면 잃는 정보가 많아지지만 연속적인 값으로 볼 때 확실하지 않은 패턴을 감지하는데 도움이 될 수 있음.
- ??? 밑에서 확인한 결과 원-핫 인코딩이 아니라 해당되는 bin의 인덱스를 반환하는데?

In [82]:
dis_layer = keras.layers.experimental.preprocessing.Discretization(bins=[-1, 0, 1, 2])

sample = np.array([[-1.5, 0.8, 3.4, 0.5], [0., 3., 1.3, 0.]])
sample2 = np.array([1,3])
dis_layer(sample2)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([3, 4])>

> 실제 모델의 전처리 층은 학습하는 동안 동결되므로 경사 하강법에 영향을 받지 않음.  
**하지만 embedding층은 학습되어야 함** 따라서 임베딩층은 전처리층 안에 놓지 말고 별도로 모델에 추가해야 함

또한 **PreprocessingStage** 클래스를 사용해 여러 전처리 층을 연결할 수 있음.  
예를 들어 입력을 정규화하고 그 다음 이산화하는 전처리 파이프라인을 만들 수 있음.  
이 파이프라인을 샘플 데이터에 적응시킨 다음(**adapt()**) 일반적인 층처럼 모델에 사용할 수 있음.
> ???? PreprocessingStage를 찾을 수 없는데??

In [95]:
cat = keras.layers.Input(shape=4, dtype=tf.float32)
normalization = keras.layers.experimental.preprocessing.Normalization()
normalization.adapt(sample)
discretization = keras.layers.experimental.preprocessing.Discretization(bins=[-1., 0., 1.0000001])(normalization(cat))
preprocessing_model = keras.models.Model(inputs=[cat], outputs=[discretization])

In [96]:
preprocessing_model.predict(sample)

array([[1, 1, 3, 2],
       [2, 2, 1, 1]])

In [90]:
normalization(sample)

<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[-1.       , -0.9999999,  1.0000002,  1.       ],
       [ 1.       ,  1.       , -1.       , -1.       ]], dtype=float32)>